In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from pynput.keyboard import Controller, Key
import pyautogui
import time

# Initialize Hand Tracker
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75
)
mp_draw = mp.solutions.drawing_utils

# Initialize Browser Control
keyboard = Controller()
screen_width, screen_height = pyautogui.size()
cap = cv2.VideoCapture(0)

prev_x, prev_y = 0, 0
smoothing_factor = 0.2  # For smoother cursor movement
prev_hand_x = None  # To detect left/right hand movement
scrolling = False  # To control continuous scrolling
scroll_direction = None  # Track scroll direction
click_timer = time.time()  # Timer to prevent rapid clicking

last_action = ""  # Variable to store the last action performed
action_display_time = time.time()  # Timer to control display duration

def detect_gesture(landmarks):
    """Detects hand gestures based on finger positions."""
    thumb, index, middle, ring, pinky = landmarks[4], landmarks[8], landmarks[12], landmarks[16], landmarks[20]

    # Distances for gesture recognition
    middle_thumb_dist = np.hypot(middle.x - thumb.x, middle.y - thumb.y)
    index_thumb_dist = np.hypot(index.x - thumb.x, index.y - thumb.y)
    pinky_thumb_dist = np.hypot(pinky.x - thumb.x, pinky.y - thumb.y)
    pinky_index_dist = np.hypot(pinky.x - index.x, pinky.y - index.y)
    pinky_middle_dist = np.hypot(pinky.x - middle.x, pinky.y - middle.y)
    pinky_ring_dist = np.hypot(pinky.x - ring.x, pinky.y - ring.y)

    fingers_extended = [landmarks[i].y < landmarks[i - 2].y for i in [8, 12, 16, 20]]

    # Gesture Conditions
    if middle_thumb_dist < 0.04:  # Middle Finger Touching Thumb (Open New Tab)
        return "open_tab"
    elif all(not f for f in fingers_extended):  # All Fingers in a Fist (Close Tab)
        return "close_tab"
    elif pinky_thumb_dist < 0.04:  # Pinky touching Thumb (Scroll Down)
        return "scroll_down"
    elif pinky_index_dist > 0.1 and pinky_middle_dist > 0.1 and pinky_ring_dist > 0.1:  # Pinky moved away from all fingers (Scroll Up)
        return "scroll_up"
    elif index_thumb_dist < 0.04:  # Index Touching Thumb (Left Click)
        return "left_click"
    
    return None

prev_gesture = None
last_scroll_time = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        continue

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x, y = int(hand_landmarks.landmark[8].x * screen_width), int(hand_landmarks.landmark[8].y * screen_height)

            # Smooth Cursor Movement
            x = int(prev_x * (1 - smoothing_factor) + x * smoothing_factor)
            y = int(prev_y * (1 - smoothing_factor) + y * smoothing_factor)
            pyautogui.moveTo(x, y, duration=0.08)

            gesture = detect_gesture(hand_landmarks.landmark)

            # Handle gestures with actions
            if gesture and gesture != prev_gesture:
                if gesture == "left_click" and time.time() - click_timer > 0.5:  # Prevent rapid clicking
                    pyautogui.click()
                    last_action = "Left Click"
                    click_timer = time.time()  # Update last click time
                elif gesture == "open_tab":
                    keyboard.press(Key.ctrl)
                    keyboard.press("t")
                    keyboard.release("t")
                    keyboard.release(Key.ctrl)
                    last_action = "New Tab Opened"
                elif gesture == "close_tab":
                    keyboard.press(Key.ctrl)
                    keyboard.press("w")
                    keyboard.release("w")
                    keyboard.release(Key.ctrl)
                    last_action = "Tab Closed"
                elif gesture == "scroll_down":
                    scrolling = True
                    scroll_direction = "down"
                    last_action = "Scrolling Down"
                elif gesture == "scroll_up":
                    scrolling = True
                    scroll_direction = "up"
                    last_action = "Scrolling Up"
                
                action_display_time = time.time()  # Reset action display timer

            elif gesture is None:  # If no gesture is detected, stop scrolling
                scrolling = False

            # Continuous scrolling with increased speed
            if scrolling and (time.time() - last_scroll_time > 0.05):  # Scroll every 0.05 seconds (Faster)
                if scroll_direction == "down":
                    pyautogui.scroll(-10)  # Faster scroll down
                elif scroll_direction == "up":
                    pyautogui.scroll(10)  # Faster scroll up
                last_scroll_time = time.time()

            # Detect hand movement for tab switching
            hand_x = hand_landmarks.landmark[0].x  # Get the x position of the wrist
            if prev_hand_x is not None:
                move_threshold = 0.08  # Define how much movement is considered a tab switch
                if prev_hand_x - hand_x > move_threshold:  # Move Left
                    keyboard.press(Key.ctrl)
                    keyboard.press(Key.shift)
                    keyboard.press(Key.tab)
                    keyboard.release(Key.tab)
                    keyboard.release(Key.shift)
                    keyboard.release(Key.ctrl)
                    last_action = "Switched Left Tab"
                elif hand_x - prev_hand_x > move_threshold:  # Move Right
                    keyboard.press(Key.ctrl)
                    keyboard.press(Key.tab)
                    keyboard.release(Key.tab)
                    keyboard.release(Key.ctrl)
                    last_action = "Switched Right Tab"

            prev_hand_x = hand_x  # Update last hand position
            prev_gesture = gesture  # Store last gesture
            prev_x = x  # Store last x position
            prev_y = y  # Store last y position
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the last action performed on the screen
    if last_action and (time.time() - action_display_time < 1.5):  # Show for 1.5 seconds
        cv2.putText(frame, last_action, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("AirNav: Fast Gesture Control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
